In [ ]:
import pathlib
import lzma
import re
import os
import datetime
import copy

import numpy as np
import pandas as pd

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys._utilities.filesystem
from prototyping import *

In [ ]:
root = pathlib.Path(r'\\physics-server\iComLogFiles\patients')
compressed_files = sorted(list(root.glob('**/*.xz')))
# compressed_files

In [ ]:
mechanical_output = root.parent.joinpath('mechanical/4299/20200116.csv')
mechanical_output.parent.mkdir(exist_ok=True)

In [ ]:
data = b""

for path in compressed_files:
    with lzma.open(path, 'r') as f:
        data += f.read()

In [ ]:
data_points = get_data_points(data)

In [ ]:
mechanical_data = {}
for data_point in data_points:
    _, result = strict_extract(data_point)
    machine_id = result['Machine ID']
    
    try:
        machine_record = mechanical_data[machine_id]
    except KeyError:
        machine_record = {}
        mechanical_data[machine_id] = machine_record
        
    timestamp = result['Timestamp']
    try:
        timestamp_record = machine_record[timestamp]
    except KeyError:
        timestamp_record = {}
        machine_record[timestamp] = timestamp_record
    
    counter = result['Counter']
    
    mlc = result['MLCX']
    mlc_a = mlc[0::2]
    mlc_b = mlc[1::2]
    width_at_cra = np.mean(mlc_b[39:41] - mlc_a[39:41])
    jaw = result['ASYMY']
    length = np.sum(jaw)
    
    timestamp_record[counter] = {
        'Energy': result['Energy'],
        'Monitor Units': result['Total MU'],
        'Gantry': result['Gantry'],
        'Collimator': result['Collimator'],
        'Table Column': result['Table Column'],
        'Table Isocentric': result['Table Isocentric'],
        'Table Vertical': result['Table Vertical'],
        'Table Longitudinal': result['Table Longitudinal'],
        'Table Lateral': result['Table Lateral'],
        'MLC distance at CRA': width_at_cra,
        'Jaw distance': length 
    }

In [ ]:
# pd.Timestamp('2020-01-16T17:08:45')

In [ ]:
table_record = pd.DataFrame(
    columns=[
        'Timestamp', 'Counter', 'Energy', 'Monitor Units', 'Gantry', 'Collimator', 'Table Column',
        'Table Isocentric', 'Table Vertical', 'Table Longitudinal',
        'Table Lateral', 'MLC distance at CRA', 'Jaw distance'
    ]
)

In [ ]:
for timestamp, timestamp_record in mechanical_data[4299].items():
    for counter, record in timestamp_record.items():
        table_record = table_record.append({
            **{
                'Timestamp': pd.Timestamp(timestamp),
                'Counter': counter
            },
            **record
        }, ignore_index=True)

In [ ]:
table_record.to_csv(mechanical_output, index=False)